# Point Process

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
output_path='/Users/Gaia_1/Desktop/Data after SS thresh 3'
list_dir=glob.glob(output_path+"/*.txt")
#for file in tqdm(list_dir):
#file='/Users/Gaia_1/Downloads/After2018-11-28T12-35-18Pop2 stimulation.h5.txt'
for file in list_dir:
    target=1
    stim=0
    if 'health' in file:
        target=0
    if 'stimulation' in file:
        stim=1
    print(file,'target',target,'stimulation',stim)
    list_neurons = np.genfromtxt(file, delimiter=',')
    df=this_poiproc(list_neurons,target,stim)
    final.append(df)

In [ ]:
df=this_single_poiproc(new_list_neurons,0,1,12)

In [ ]:
new_length = len(list_neurons) // 3

# Create a new NumPy array to store pairs of arrays
new_list_neurons = np.empty(new_length, dtype=object)

# Iterate through the original array, combining every pair of arrays
for i in range(0, new_length * 3, 3):
    new_list_neurons[i // 3] = np.array([list_neurons[i], list_neurons[i + 1],list_neurons[i + 2]])

# Print the new array
print(len(new_list_neurons))

# Functions

In [ ]:
def this_Bayesian_mixture_model(ISI_data):
    import scipy.stats as st
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.03,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.000001,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0.09,upper=0.4)
        sigma2 = pm.Uniform('sigma2',lower=0.01,upper=0.7)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)
        
        w = pm.Dirichlet('w', a=np.array([1., 1.]))
        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2], observed=ISI_data)

    map_estimate = pm.find_MAP(model=model)
    d= np.linspace(0.00, 1, len(ISI_data))
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    
    mu1=map_estimate['mu1']
    mu2=map_estimate['mu2']
    
    #lam1=map_estimate['lam1']
    lam1=0.008
    sigma2=map_estimate['sigma2']
    
    w1=map_estimate['w1']
    w2=map_estimate['w2']
    
    pdf = w1*st.invgauss.pdf(d, mu1/lam1, scale = lam1) + w2*st.norm.pdf(d, mu2, sigma2)
    bins = np.arange(0, 0.5, 1e-3)
    
    plt.hist(ISI_data, bins, color='orange', alpha=0.7, label='Histogram',density=True)
    plt.plot(d, pdf, color='blue', label='PDF')
    #plt.xlim(0, 0.3)
    plt.show()

    cdf_2gauss = lambda x: w1 * st.invgauss.cdf(x,mu1/lam1, scale = lam1) + w2 * st.norm.cdf(x, mu2, sigma2)
    ks_score=st.ks_1samp(ISI_data[ISI_data>0],  cdf_2gauss, method = 'asymp')
    print('ks score: ',ks_score)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']

    
    del map_estimate['w']

    print(map_estimate)
    return map_estimate


In [ ]:
def this_poiproc(neurons,target,stim):
    from scipy.stats import ks_2samp
    dataframe = pd.DataFrame()
    #counter_net=1
    counter=0
    #for net in list_dir_ok:
    #print(counter_net,') ',net)
    #counter_net+=1
    list_neurons = neurons #np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in tqdm(list_neurons):
        neuron=neuron[neuron>0*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>10:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        map_estimate = this_Bayesian_mixture_model(ISI_healthy)
        map_estimate['Target']=target
        map_estimate['Stimulation']=stim
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)
    #ks_2samp(lista_samples,ISI_healthy,mode = 'asymp')
    return dataframe


In [ ]:
def this_single_poiproc(neurons,target,stim,which):
    from scipy.stats import ks_2samp
    dataframe = pd.DataFrame()
    #counter_net=1
    #list_neurons = neurons #np.genfromtxt(net, delimiter=',')
    print('Original number of neurons: ',len(neurons))
    neuron=list_neurons[which]
    neuron=neuron[neuron>0*10000]
    neuron=neuron[neuron<200*10000]
    print('  Neuron with ',neuron.shape[0],'spikes')
    if neuron.shape[0]<10:
        print('    Excluded neuron with n spikes = ',neuron.shape[0])

    ISI_healthy = np.diff(neuron)/10000
    map_estimate = this_Bayesian_mixture_model(ISI_healthy)
    map_estimate['Target']=target
    map_estimate['Stimulation']=stim
    df = pd.DataFrame.from_dict(map_estimate,orient='index')
    dataframe = pd.concat([dataframe,df],axis = 1)
    print('Target = ',target)
    #ks_2samp(lista_samples,ISI_healthy,mode = 'asymp')
    return dataframe


def this_Bayesian_mixture_model(ISI_data):
    import scipy.stats as st
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.03,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.000001,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0.02,upper=0.04)
        sigma2 = pm.Uniform('sigma2',lower=0.01,upper=0.7)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.04,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.01,upper=0.7)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)
        
        w = pm.Dirichlet('w', a=np.array([1., 0.3, 0.3]))
        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)

        '''
        if counter==9:
            step = pm.NUTS(target_accept=0.9)
            trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)
            ppc_trace = pm.sample_posterior_predictive(trace,model=model)
            bins = np.arange(0, .5, 1e-3) 
            plt.figure (figsize=(14,10))
            hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
            #plt.axis([-0.01,0.13,0,160])
            a= plt.hist(ISI_data,bins)
            plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3)
            plt.show()
        '''
        
    map_estimate = pm.find_MAP(model=model)
    d= np.linspace(0.00, 1, len(ISI_data))
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    
    mu1=map_estimate['mu1']
    mu2=map_estimate['mu2']
    mu3=map_estimate['mu3']
    
    lam1=map_estimate['lam1']
    sigma2=map_estimate['sigma2']
    sigma3=map_estimate['sigma3']
    
    w1=map_estimate['w1']
    w2=map_estimate['w2']
    w3=map_estimate['w3']
    
    pdf = w1*st.invgauss.pdf(d, mu1/lam1, scale = lam1) + w2*st.norm.pdf(d, mu2, sigma2) + w3*st.norm.pdf(d, mu3, sigma3)
    bins = np.arange(0, 0.5, 1e-3)
    
    plt.hist(ISI_data, bins, color='orange', alpha=0.7, label='Histogram',density=True)
    plt.plot(d, pdf, color='blue', label='PDF')
    plt.xlim(0, 0.5)
    plt.show()

    cdf_2gauss = lambda x: w1 * st.invgauss.cdf(x,mu1/lam1, scale = lam1) + w2 * st.norm.cdf(x, mu2, sigma2) + w3*st.norm.cdf(x, mu3, sigma3)
    ks_score=st.ks_1samp(ISI_data[ISI_data>0],  cdf_2gauss, method = 'asymp')
    print('ks score: ',ks_score)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']

    
    del map_estimate['w']

    print(map_estimate)
    return map_estimate


def this_Bayesian_mixture_model(ISI_data):
    import scipy.stats as st
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.03,upper=0.04)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.4)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0.02,upper=0.04)
        sigma2 = pm.Uniform('sigma2',lower=0.01,upper=0.7)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.04,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.01,upper=0.7)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)
        
        mu4 = pm.Uniform('mu4',lower=0.04,upper=0.05)
        lam4 = pm.Uniform('lam4',lower=0.0001,upper=0.04)
        obs4 = pm.Wald.dist(mu=mu4,lam=lam4)        
        
        w = pm.Dirichlet('w', a=np.array([0.5, 1., 1e-10, 0.5]))
        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3, obs4], observed=ISI_data)

        '''
        if counter==9:
            step = pm.NUTS(target_accept=0.9)
            trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)
            ppc_trace = pm.sample_posterior_predictive(trace,model=model)
            bins = np.arange(0, .5, 1e-3) 
            plt.figure (figsize=(14,10))
            hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
            #plt.axis([-0.01,0.13,0,160])
            a= plt.hist(ISI_data,bins)
            plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3)
            plt.show()
        '''
        
    map_estimate = pm.find_MAP(model=model)
    d= np.linspace(0.00, 1, len(ISI_data))
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    map_estimate['w4'] = map_estimate['w'][3]
    
    mu1=map_estimate['mu1']
    mu2=map_estimate['mu2']
    mu3=map_estimate['mu3']
    mu4=map_estimate['mu4']
    
    lam1=map_estimate['lam1']
    sigma2=map_estimate['sigma2']
    sigma3=map_estimate['sigma3']
    lam4=map_estimate['lam4']
    
    w1=map_estimate['w1']
    w2=map_estimate['w2']
    w3=map_estimate['w3']
    w4=map_estimate['w4']
    
    pdf = w1*st.invgauss.pdf(d, mu1/lam1, scale = lam1) + w2*st.norm.pdf(d, mu2, sigma2) + w3*st.norm.pdf(d, mu3, sigma3)+w4*st.invgauss.pdf(d, mu4/lam4, scale = lam4)
    bins = np.arange(0, 0.5, 1e-3)
    
    plt.hist(ISI_data, bins, color='orange', alpha=0.7, label='Histogram',density=True)
    plt.plot(d, pdf, color='blue', label='PDF')
    #plt.xlim(0, 0.5)
    plt.show()

    cdf_2gauss = lambda x: w1 * st.invgauss.cdf(x,mu1/lam1, scale = lam1) + w2 * st.norm.cdf(x, mu2, sigma2) + w3*st.norm.cdf(x, mu3, sigma3)+w4 * st.invgauss.cdf(x,mu4/lam4, scale = lam4)
    ks_score=st.ks_1samp(ISI_data[ISI_data>0],  cdf_2gauss, method = 'asymp')
    print('ks score: ',ks_score)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    del map_estimate['mu4_interval__']
    del map_estimate['lam4_interval__']

    
    del map_estimate['w']

    print(map_estimate)
    return map_estimate
